In [11]:
import numpy as np
import os
import csv
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
from torch.utils.data import DataLoader, Dataset

df1 = pd.read_csv('train.csv')
print(df1.shape)
print(df1.count())
print(df1.info())
print(df1.describe())
class TitanicSet(Dataset) :
    def __init__(self, path, model = 'train') :
        super(TitanicSet, self).__init__()
        self.model = model
        with open(path, 'r') as file :
            data = list(csv.reader(file))
            data = np.array(data[1 : ])
        if model == 'test' :
            return torch.tensor(self.data[ : , 1 : ])
        else :
            target = data[ : , 1]
            data = data[ : , 2 : ]
            train_index = []
            val_index = []
            for i in range(data.shape[0]) :
                if i % 10 == 0 :
                    val_index.append(i)
                else :
                    train_index.append(i)
            if model == 'val' :
                self.data = torch.tensor(data[train_index, : ])
                self.target = torch.LongTensor(target[train_index])
            else :
                self.data = torch.tensor(data[val_index, : ])
                self.target = torch.LongTensor(target[val_index])
        def __getitem__(self, item) :
            if self.model == 'test' :
                return self.data[item]
            else :
                return self.data[item], self.target[item]
        def __len__(self) :
            return len(self.data)
class MyModel(nn.Module) :
    def __init__(self) :
        super(MyModel, self).__init__()
        self.net = nn.Sequential(nn.Linear(891, 512),
                                nn.ReLU(),
                                nn.Linear(512, 256),
                                nn.ReLU(),
                                nn.Linear(256, 128),
                                nn.ReLU(),
                                nn.Linear(128, 64),
                                nn.ReLU(),
                                nn.Linear(64, 2))
    def forward(self, input) :
        return self.net(input)

train_set = TitanicSet('train.csv', model = 'train')
val_set = TitanicSet('train.csv', model = 'val')
test_set = TitanicSet('test.csv', model = 'test')

batch_size = 64
train_loader = DataLoader(train_set, batch_size=batch_size, shuffle=True, drop_last=False)
val_loader = DataLoader(val_set, batch_size=batch_size, shuffle=False, drop_last=False)
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle=False, drop_last=False)

model = MyModel().cuda()
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()
max_epoch = 1000

for i in range(max_epoch) :
    train_loss = 0.0
    train_acc = 0.0
    val_losses = []
    train_losses = []
    min_mse = 1000
    model.train()
    for x, label in train_loader :
        x, label = x.cuda(), label.cuda()
        optimizer.zero_grad()
        output = model(x)
        loss = criterion(output, label)
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        _, pred = torch.max(output, 1)
        num_correct = (pred == label).sum().item()
        acc = num_correct / x.shape[0]
        train_acc += acc
    total_loss = []
    with torch.no_grad() :
        model.eval()
        for x, label in val_loader :
            output = model(x)
            loss = criterion(output, label)
            total_loss.append(loss.item())
        val_loss = sum(total_loss) / len(total_loss)
    if val_loss < min_mse :
        min_mse = val_loss
        torch.save(model.state_dicta(), 'mymodel.pth')
    print('epoch : {:02d}, train_loss : {}, train_acc : {}'.format(epoch + 1, 
                                                               train_loss / len(train_loader), 
                                                               train_acc  / len(train_loader)))
    train_losses.append(train_loss / len(train_loader))
    val_acces.append(train_acc  / len(train_loader))

del model
bestmodel = MyModel().cuda()
ckpt = torch.load('mymodel.pth', map_location= "cpu")
bestmodel.load_state_dict(ckpt)

with torch.no_grad() :
    output = []
    bestmodel.eval()
    for x in test_loader :
        x = x.cuda()
        out = bestmodel(x)
        _, pred = torch.max(out, 1)
        for i in pred.cpu().numpy() :
            output.append(i)

with open('predictions.csv', 'w') as file :
    writer = csv.writer(file)
    writer.writerow(['PassengerId', 'Survived'])
    for id, label in enumerate(output) :
        writer.writerow([id + 892, label])

(891, 12)
PassengerId    891
Survived       891
Pclass         891
Name           891
Sex            891
Age            714
SibSp          891
Parch          891
Ticket         891
Fare           891
Cabin          204
Embarked       889
dtype: int64
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83

TypeError: can't convert np.ndarray of type numpy.str_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint8, and bool.